In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
import warnings
from datetime import date

warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='dark')
# Mon PC n'est pas capable de faire tourner le Dbscan sur 100% des
# données, je selectionne donc le pourcentage que j'utiliserais
pourcentage_utilise = 50

In [2]:
# data1 = nom des produits
data1 = pd.read_csv('product_category_name_translation.csv')

# data2 = liste des vendeurs
data2 = pd.read_csv('olist_sellers_dataset.csv')

# data3 = liste des produits (sans prix)
data3 = pd.read_csv('olist_products_dataset.csv')

# data4 = liste des commandes avec id clients (sans prix)
data4 = pd.read_csv('olist_orders_dataset.csv')

# data5 = liste des review
data5 = pd.read_csv('olist_order_reviews_dataset.csv')

# data6 = liste des commandes avec prix (sans id clients)
data6 = pd.read_csv('olist_order_items_dataset.csv')

# data7 = bdd villes de livraisons
data7 = pd.read_csv('olist_geolocation_dataset.csv')

# data8 = liste des clients
data8 = pd.read_csv('olist_customers_dataset.csv')

In [3]:
data1

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
...,...,...
66,flores,flowers
67,artes_e_artesanato,arts_and_craftmanship
68,fraldas_higiene,diapers_and_hygiene
69,fashion_roupa_infanto_juvenil,fashion_childrens_clothes


In [4]:
data2

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [5]:
data3

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [6]:
data4.drop(
    columns=[
        'order_estimated_delivery_date',
        'order_delivered_customer_date'])

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34
...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03


In [7]:
data5.drop(
    columns=[
        'review_comment_title',
        'review_id',
        'review_comment_title'])

,order_id,review_score,review_comment_message,review_creation_date,review_answer_timestamp
0,73fc7af87114b39712e6da79b0a377eb,4,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,a548910a1c6147796b98fdf73dbeba33,5,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,658677c97b385a9be170737859d3511b,5,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,5,NaN,2018-07-07 00:00:00,2018-07-14 17:18:30
99220,22ec9f0669f784db00fa86d035cf8602,5,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42
99221,55d4004744368f5571d1f590031933e4,5,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99222,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13


In [8]:
data6.drop(columns=['order_item_id'])

,order_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [9]:
data7

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [10]:
data8

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [11]:
# On lie les data 4 et 6 pour avoir un dataframme comprenant les orders fait par les clients avec l'id client, et le prix.
# On vire les colonnes qui ne nous interessent pas, puis on garde que les statuts 'délivrés' pour notre étude.
# On vire finalement les quelques valeurs manquantes (moins de 100 sur les
# 100 000) de dates de livraisons.

data46 = data4.set_index('order_id').join(
    data6.set_index('order_id')).reset_index()
data46.drop(columns=['order_purchase_timestamp',
                     'order_approved_at',
                     'order_delivered_carrier_date',
                     'order_estimated_delivery_date',
                     'shipping_limit_date',
                     'freight_value'],
            inplace=True)
data46 = data46.loc[data46['order_status'] ==
                    'delivered'].drop(columns=['order_status'])
data46 = data46.loc[~data46['order_delivered_customer_date'].isna()]

In [12]:
# Pour un même ordre d'achat, on remarque qu'il ont séparés les pack d'articles en plusieurs lignes. Ces 4 lignes corresponde
# A un achat de 4 fois le même produit. On rassemble ce genre de
# particularité en une unique ligne.

data46.loc[data46['order_id'] == 'fffb9224b6fc7c43ebb0904318b10b5f']

,order_id,customer_id,order_delivered_customer_date,order_item_id,product_id,seller_id,price
113415,fffb9224b6fc7c43ebb0904318b10b5f,4d3abb73ceb86353aeadbe698aa9d5cb,2017-11-17 19:41:42,1.0,43423cdffde7fda63d0414ed38c11a73,b1fc4f64df5a0e8b6913ab38803c57a9,55.0
113416,fffb9224b6fc7c43ebb0904318b10b5f,4d3abb73ceb86353aeadbe698aa9d5cb,2017-11-17 19:41:42,2.0,43423cdffde7fda63d0414ed38c11a73,b1fc4f64df5a0e8b6913ab38803c57a9,55.0
113417,fffb9224b6fc7c43ebb0904318b10b5f,4d3abb73ceb86353aeadbe698aa9d5cb,2017-11-17 19:41:42,3.0,43423cdffde7fda63d0414ed38c11a73,b1fc4f64df5a0e8b6913ab38803c57a9,55.0
113418,fffb9224b6fc7c43ebb0904318b10b5f,4d3abb73ceb86353aeadbe698aa9d5cb,2017-11-17 19:41:42,4.0,43423cdffde7fda63d0414ed38c11a73,b1fc4f64df5a0e8b6913ab38803c57a9,55.0


In [13]:
# On calcule le prix total
data_agrege = data46.groupby(by='order_id').sum().reset_index()
data_agrege = data_agrege.rename(columns={'price': 'price_agrege'})
data_agrege = data_agrege.drop(columns='order_item_id')
data46 = data46.set_index('order_id').join(
    data_agrege.set_index('order_id')).reset_index()

# On garde la donnée du nombre de produit acheté
data46 = data46.set_index('order_item_id')
data46 = data46.drop_duplicates(keep='last')
data46 = data46.reset_index()
data46.drop(columns=['price'], inplace=True)
data46.rename(
    columns={
        'price_agrege': 'price',
        'order_item_id': 'number_of_product_buy'},
    inplace=True)

In [14]:
# Resultat :
data46.loc[data46['order_id'] == 'fffb9224b6fc7c43ebb0904318b10b5f']

,number_of_product_buy,order_id,customer_id,order_delivered_customer_date,product_id,seller_id,price
100181,4.0,fffb9224b6fc7c43ebb0904318b10b5f,4d3abb73ceb86353aeadbe698aa9d5cb,2017-11-17 19:41:42,43423cdffde7fda63d0414ed38c11a73,b1fc4f64df5a0e8b6913ab38803c57a9,220.0


In [15]:
# Dans le data 5 des review, le même problème d'un review_id pour plusieurs order_id apparait. On supprime pour n'en garder qu'un
# par commande passés.
data5.loc[data5['review_id'] == '308316408775d1600dad81bd3184556d']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
4545,308316408775d1600dad81bd3184556d,27e5ed2ecefbe0bcc9ae0c9b26b763c3,5,NaN,Ajudem a rastrear uma das 3 prateleiras que fa...,2017-09-07 00:00:00,2017-09-11 09:58:09
5719,308316408775d1600dad81bd3184556d,3fe4dbcdb046a475dbf25463c1ca78bd,5,NaN,Ajudem a rastrear uma das 3 prateleiras que fa...,2017-09-07 00:00:00,2017-09-11 09:58:09
53661,308316408775d1600dad81bd3184556d,8b3c2785144e72ccba9b0213f0f1cd1e,5,NaN,Ajudem a rastrear uma das 3 prateleiras que fa...,2017-09-07 00:00:00,2017-09-11 09:58:09


In [16]:
# On règle ça :
data5 = data5.set_index('order_id')
data5 = data5.drop_duplicates(keep='last')
data5 = data5.reset_index()
data5.loc[data5['review_id'] == '308316408775d1600dad81bd3184556d']

,order_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
53012,8b3c2785144e72ccba9b0213f0f1cd1e,308316408775d1600dad81bd3184556d,5,NaN,Ajudem a rastrear uma das 3 prateleiras que fa...,2017-09-07 00:00:00,2017-09-11 09:58:09


In [17]:
# Ajoutons mainteant les reviews fait par les clients (data5). On ne
# gardera que les dates d'envois, de réponse, et la note proposé.

data456 = data46.set_index('order_id').join(
    data5.set_index('order_id')).reset_index()
data456.drop(columns=['review_id',
                      'review_comment_title',
                      'review_comment_message'],
             inplace=True)

# Le data3 propose des catégories prédéfinies pour les différents produits. On ajoute cette information, en retirant
# Les 7 dernière colones inutiles du data3

data4563 = data456.set_index('product_id').join(
    data3.set_index('product_id')).reset_index()
data4563.drop(columns=[i for i in data4563.columns[-7:]], inplace=True)

# On ajoute désormais deux informations géographique : l'état et la ville du client.
# Le data8 contient également l'id unique par client. Cet id corresponds à
# un client, et ne change pas au cours du temps.


data45638 = data4563.set_index('customer_id').join(
    data8.set_index('customer_id')).reset_index()
data45638.drop(
    columns=[
        'customer_zip_code_prefix',
        'customer_id',
        'customer_city'],
    inplace=True)

In [18]:
# De sorte à rendre les informations datés utiles, créeons une fonction qui calculer le nombre de jours entre le 1 janvier 2015
# Et la date en question


d = {
    '1': 31,
    '2': 28,
    '3': 31,
    '4': 30,
    '5': 31,
    '6': 30,
    '7': 31,
    '8': 31,
    '9': 30,
    '10': 31,
    '11': 30,
    '12': 31}


def nombre_heure_mois(string):
    heures = 0
    for i in range(1, int(string) + 1):
        heures += d[str(i)]
    return heures


def date_to_int(string):
    today = str(date.today())
    date_int = (int(string[0:4]) - int(today[0:4])) * 365 + nombre_heure_mois(
        string[5:7]) - nombre_heure_mois(today[5:7]) + int(string[8:10]) - int(today[8:10])
    return date_int


data45638['order_delivered_customer_date'] = data45638['order_delivered_customer_date'].apply(
    date_to_int)
data45638.loc[
    ~data45638['review_creation_date'].isna(), 'review_creation_date'] = data45638.loc[
    ~data45638['review_creation_date'].isna(), 'review_creation_date'].apply(date_to_int)
data45638.loc[
    ~data45638['review_answer_timestamp'].isna(), 'review_answer_timestamp'] = data45638.loc[
    ~data45638['review_answer_timestamp'].isna(), 'review_answer_timestamp'].apply(date_to_int)

# On a désormais plus besoin du product_id, du order_id, ainsi que du
# seller_id.
data45638.drop(columns=['product_id', 'order_id', 'seller_id'], inplace=True)
data45638['order_delivered_customer_date'] = round(
    data45638['order_delivered_customer_date'], 2)

In [19]:
# on crée maintenant la réactivité du client au sondage et on fill les NaN
# par la valeur moyenne.

data45638['review_reactivity'] = data45638['review_answer_timestamp'] - \
    data45638['review_creation_date']
data45638['review_reactivity'] = data45638['review_reactivity'].astype('float')
data45638.loc[data45638['review_reactivity'].isna(),
              'review_reactivity'] = data45638['review_reactivity'].mean()
data45638.loc[data45638['review_score'].isna(),
              'review_score'] = data45638['review_score'].mean()
data45638.round(2)
data45638.drop(
    columns=[
        'review_answer_timestamp',
        'review_creation_date'],
    inplace=True)

# On récupère maintenant les noms de produit en anglais (je ne parle pas
# un mot d'espagnol)
data45638 = data45638.set_index('product_category_name').join(
    data1.set_index('product_category_name')).reset_index()
data45638.drop(columns=['product_category_name'], inplace=True)
data45638.loc[data45638['product_category_name_english'].isna(),
              'product_category_name_english'] = "no_value"

In [20]:
# On va agglomérer les catégories de produits dans des catégories plus générales, et également transformer
# l'information de la région en PIB/Hab de ladite région :

industry_and_building = [
    'agro_industry_and_commerce', 'industry_commerce_and_business', 'signaling_and_security', 'security_and_services',
    'auto', 'home_construction', 'air_conditioning', 'construction_tools_construction', 'costruction_tools_tools',
    'construction_tools_lights', 'costruction_tools_garden',
    'construction_tools_safety', 'garden_tools', 'furniture_mattress_and_upholstery', 'kitchen_dining_laundry_garden_furniture',
]
everyday_needs = [
    'food', 'food_drink', 'baby', 'drinks', 'health_beauty', 'fashion_bags_accessories',
    'fashion_shoes', 'fashion_sport', 'fashio_female_clothing',
    'fashion_childrens_clothes', 'fashion_male_clothing',
    'fashion_underwear_beach', 'diapers_and_hygiene', 'la_cuisine', 'market_place', 'pet_shop',
    'small_appliances_home_oven_and_coffee', 'housewares']

art_and_hobby = [
    'art', 'arts_and_craftmanship', 'toys', 'bed_bath_table',
    'cds_dvds_musicals', 'cine_photo',
    'home_appliances_2', 'small_appliances', 'sports_leisure', 'musical_instruments',
    'books_imported', 'books_general_interest', 'books_technical', 'flowers', 'music', 'cool_stuff',
    'party_supplies', 'christmas_supplies', 'luggage_accessories', 'watches_gifts', 'perfumery']

electronics = [
    'computers', 'telephony', 'fixed_telephony', 'electronics', 'consoles_games', 'dvds_blu_ray',
    'audio', 'computers_accessories', 'tablets_printing_image']

home_furniture = [
    'home_confort', 'home_comfort_2', 'home_appliances', 'furniture_decor', 'office_furniture',
    'furniture_bedroom', 'furniture_living_room', 'stationery']


def changer_product_categorie(string):
    if string == 'no_value':
        return 'Other'
    if industry_and_building.count(string) > 0:
        return 'industry_and_building'
    if everyday_needs.count(string) > 0:
        return 'everyday_needs'
    if art_and_hobby.count(string) > 0:
        return 'art_and_hobby'
    if electronics.count(string) > 0:
        return 'electronics'
    if home_furniture.count(string) > 0:
        return 'home_furniture'


d = {'Liste_region': ['SP', 'RJ', 'MG', 'RS', 'PR', 'SC', 'BA', 'DF', 'GO', 'PE', 'CE', 'PA', 'MT',
                      'ES', 'MS', 'AM', 'MA', 'RN', 'AL', 'PB', 'SE', 'PI', 'AC', 'TO', 'RO', 'AP', 'RR'],
     'PIB_par_hab_en_dollar': [8260, 7745, 5274, 7669, 7202, 7952, 3255, 15770, 5482, 4243, 3728, 3705, 7708,
                               5937, 6776, 4892, 2768, 4125, 4623, 2850, 3610, 1646, 4473, 2660, 1805, 2023, 2334]}
revenu_par_region_bresil = pd.DataFrame(d)
data45638 = data45638.set_index('customer_state').join(
    revenu_par_region_bresil.set_index('Liste_region')).reset_index()
data45638.drop(columns=['index'], inplace=True)


data45638['product_category_name_english'] = data45638['product_category_name_english'].apply(
    changer_product_categorie)
dummies = pd.get_dummies(
    data45638['product_category_name_english'],
    columns=['product_category_name_english'])
data45638 = data45638.join(dummies)
for i in dummies.columns:
    data45638[i] = data45638[i].astype('float')
data45638.drop(columns=['product_category_name_english'], inplace=True)
data45638.describe()

,number_of_product_buy,order_delivered_customer_date,price,review_score,review_reactivity,PIB_par_hab_en_dollar,Other,art_and_hobby,electronics,everyday_needs,home_furniture,industry_and_building
count,100479.000000,100479.000000,100479.000000,100479.000000,100479.000000,100479.000000,100479.000000,100479.000000,100479.000000,100479.000000,100479.000000,100479.000000
mean,1.151315,-1956.897879,139.850037,4.128103,2.575401,7248.697589,0.014182,0.377751,0.153047,0.231123,0.119328,0.104569
std,0.563892,152.238065,210.634165,1.300751,9.724676,2071.840534,0.118242,0.484827,0.360034,0.421553,0.324176,0.305999
min,1.000000,-2416.000000,0.850000,1.000000,-2.000000,1646.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,-2068.000000,47.490000,4.000000,1.000000,5937.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,-1939.000000,89.000000,5.000000,1.000000,7745.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,-1835.000000,152.000000,5.000000,3.000000,8260.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,21.000000,-1680.000000,13440.000000,5.000000,518.000000,15770.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [21]:
data45638.isna().sum()

number_of_product_buy            0
order_delivered_customer_date    0
price                            0
review_score                     0
customer_unique_id               0
review_reactivity                0
PIB_par_hab_en_dollar            0
Other                            0
art_and_hobby                    0
electronics                      0
everyday_needs                   0
home_furniture                   0
industry_and_building            0
dtype: int64

In [22]:
# Une ou plusieurs valeur abérantes à gerer sur la réactivité (date de retour client plus tôt dans le temps que la date d'envoi
# du sondage)
data45638.loc[data45638['review_reactivity'] < 0, 'review_reactivity'] = 0

In [23]:
# On va maintenant aglomerer les données pour qu'une ligne corresponde a un client et l'ensemble de ses achat dans le temps
# à l'aide de plusieurs fonctions :

def aglomerer_prix_nombre_produit_type_produit(dataa):
    data = dataa.copy(deep=True)
    data_agrege = data.groupby(by='customer_unique_id').sum().reset_index()
    data_agrege = data_agrege.rename(columns={'price': 'price_agrege',
                                              'number_of_product_buy': 'total_number_product_bought',
                                              'Other': 'Other_aglogere',
                                              'art_and_hobby': 'art_and_hobby_aglomere',
                                              'electronics': 'electronics_aglomere',
                                              'everyday_needs': 'everyday_needs_aglomere',
                                              'home_furniture': 'home_furniture_aglomere',
                                              'industry_and_building': 'industry_and_building_aglomere'})

    data_agrege = data_agrege[['customer_unique_id',
                               'price_agrege',
                               'total_number_product_bought',
                               'Other_aglogere',
                               'art_and_hobby_aglomere',
                               'electronics_aglomere',
                               'everyday_needs_aglomere',
                               'home_furniture_aglomere',
                               'industry_and_building_aglomere']]

    data.drop(columns=['Other',
                       'art_and_hobby',
                       'electronics',
                       'everyday_needs',
                       'home_furniture',
                       'industry_and_building',
                       'price',
                       'number_of_product_buy'], inplace=True)

    return data.set_index('customer_unique_id').join(data_agrege.set_index('customer_unique_id')).reset_index().rename(
        columns={'price_agrege': 'price',
                 'Other_aglogere': 'Other',
                 'art_and_hobby_aglomere': 'art_and_hobby',
                 'electronics_aglomere': 'electronics',
                 'everyday_needs_aglomere': 'everyday_needs',
                 'home_furniture_aglomere': 'home_furniture',
                 'industry_and_building_aglomere': 'industry_and_building'})


def aglomerer_review_score_reaction_and_PIB(dataa):
    data = dataa.copy(deep=True)
    data_agrege = data.groupby(by='customer_unique_id').mean().reset_index()
    data_agrege = data_agrege.rename(columns={'review_score': 'review_score_agrege',
                                              'review_reactivity': 'review_reactivity_agrege',
                                              'PIB_par_hab_en_dollar': 'PIB_par_hab_en_dollar_agrege'})
    data_agrege = data_agrege[['customer_unique_id', 'review_score_agrege',
                               'review_reactivity_agrege',
                               'PIB_par_hab_en_dollar_agrege']]
    data.drop(
        columns=[
            'review_score',
            'review_reactivity',
            'PIB_par_hab_en_dollar'],
        inplace=True)
    return data.set_index('customer_unique_id').join(data_agrege.set_index('customer_unique_id')).reset_index().rename(
        columns={'review_score_agrege': 'review_score',
                 'review_reactivity_agrege': 'review_reactivity',
                 'PIB_par_hab_en_dollar_agrege': 'PIB_par_hab_en_dollar'})


def aglomerer_number_of_buy(dataa):
    data = dataa.copy(deep=True)
    data_agrege = data.groupby(by='customer_unique_id').count().reset_index()
    data_agrege = data_agrege.rename(
        columns={'order_delivered_customer_date': 'number_of_order_made'})
    data_agrege = data_agrege[['customer_unique_id', 'number_of_order_made']]
    return data.set_index('customer_unique_id').join(
        data_agrege.set_index('customer_unique_id')).reset_index()

def get_last_client_appearance(dataa):
    data = dataa.copy(deep=True)
    data_agrege = data.groupby(by='customer_unique_id').max().reset_index()
    data_agrege = data_agrege.rename(columns={'order_delivered_customer_date': 'last_seen',
                                              })
    data_agrege = data_agrege[['customer_unique_id', 'last_seen']]
    return data.set_index('customer_unique_id').join(
        data_agrege.set_index('customer_unique_id')).reset_index()

# Cette dernière fonction aglomère de sorte qu'un client soit représenté
# par une unique ligne dans le dataset :


def one_line_is_one_client(data):
    data_copy = data.copy(deep=True)
    data_copy = aglomerer_prix_nombre_produit_type_produit(data_copy)
    data_copy = aglomerer_review_score_reaction_and_PIB(data_copy)
    data_copy = aglomerer_number_of_buy(data_copy)
    data_copy = get_last_client_appearance(data_copy)
    data_copy.drop(columns=['order_delivered_customer_date'], inplace=True)
    data_copy.drop_duplicates(inplace=True)
    data_copy = data_copy.reindex(['customer_unique_id',
                                   'price', 'total_number_product_bought',
                                   'number_of_order_made', 'last_seen', 'review_score',
                                   'review_reactivity', 'PIB_par_hab_en_dollar',
                                   'Other', 'art_and_hobby', 'electronics',
                                   'everyday_needs', 'home_furniture',
                                   'industry_and_building'], axis='columns')
    return data_copy

In [38]:
# On obtient ainsi pour chaque client des lignes identiques sauf pour la date de livraison (seul renseignement temporel restant)
# Je ne peut pas faire tourner les algo sur mon PC avec l'ensemble des données. On en garde donc qu'un pourcentage pour la suite
# De l'étude qui sera appliquable sur un meilleur PC sur l'ensemble des
# données (on garde les données les plus récentes).
total_valeurs_a_prendre = int(data45638.shape[0] * pourcentage_utilise / 100)
data45638 = data45638.sort_values(
    by='order_delivered_customer_date', ascending=False).iloc[:total_valeurs_a_prendre, :]
data45638.to_csv('data_avec_date.csv')

In [39]:
# Avant agglomération des valeurs :
data45638.loc[data45638['customer_unique_id']
              == '8d50f5eadf50201ccdcedfb9e2ac8455']

,number_of_product_buy,order_delivered_customer_date,price,review_score,customer_unique_id,review_reactivity,PIB_par_hab_en_dollar,Other,art_and_hobby,electronics,everyday_needs,home_furniture,industry_and_building
73179,1.0,-1734,23.40,5.0,8d50f5eadf50201ccdcedfb9e2ac8455,0.0,8260,0.0,0.0,0.0,0.0,0.0,1.0
77093,1.0,-1734,72.90,5.0,8d50f5eadf50201ccdcedfb9e2ac8455,1.0,8260,0.0,1.0,0.0,0.0,0.0,0.0
78184,1.0,-1748,26.99,5.0,8d50f5eadf50201ccdcedfb9e2ac8455,1.0,8260,0.0,1.0,0.0,0.0,0.0,0.0
77730,1.0,-1763,99.00,5.0,8d50f5eadf50201ccdcedfb9e2ac8455,1.0,8260,0.0,1.0,0.0,0.0,0.0,0.0
80484,1.0,-1779,22.99,5.0,8d50f5eadf50201ccdcedfb9e2ac8455,1.0,8260,0.0,0.0,0.0,1.0,0.0,0.0
77687,1.0,-1826,51.80,5.0,8d50f5eadf50201ccdcedfb9e2ac8455,0.0,8260,0.0,1.0,0.0,0.0,0.0,0.0
78585,1.0,-1827,45.99,5.0,8d50f5eadf50201ccdcedfb9e2ac8455,0.0,8260,0.0,1.0,0.0,0.0,0.0,0.0


In [40]:
# Après :
df = one_line_is_one_client(data45638)
df.to_csv('df_final')
df.loc[df['customer_unique_id'] == '8d50f5eadf50201ccdcedfb9e2ac8455']

,customer_unique_id,price,total_number_product_bought,number_of_order_made,last_seen,review_score,review_reactivity,PIB_par_hab_en_dollar,Other,art_and_hobby,electronics,everyday_needs,home_furniture,industry_and_building
27660,8d50f5eadf50201ccdcedfb9e2ac8455,343.07,7.0,7,-1734,5.0,0.571429,8260.0,0.0,5.0,0.0,1.0,0.0,1.0


In [27]:
# Pour finir, on divise les données en périodes : On crée plusieurs intervalles de la forme [t0, tk], [t0, tk+1]...
# Le dernier de ces intervalle est donc l'ensemble des données, et cette division permettra une étude de l'évolution du
# clustering fait par la suite

def division_periode(data, nb_periode, periode):
    date0 = data45638['order_delivered_customer_date'].min()
    datefinale = data45638['order_delivered_customer_date'].max()
    intervalles = np.append(np.arange(date0,
                                      datefinale,
                                      (datefinale - date0) / nb_periode),
                            data45638['order_delivered_customer_date'].max())
    min_date = intervalles[0]
    max_date = intervalles[periode + 1]
    return data.loc[(data['order_delivered_customer_date'] >= min_date) & (
        data['order_delivered_customer_date'] <= max_date)]


def creation_data_periode(data, nb_periode):
    liste_data = []
    for k in range(nb_periode):
        liste_data.append(
            one_line_is_one_client(
                division_periode(
                    data, nb_periode, k)))
    return liste_data


def create_dataframe_per_periode(nb_periode):
    liste_data = creation_data_periode(data45638, nb_periode)
    for k in range(nb_periode):
        liste_data[k].to_csv('df_' + str(nb_periode) + '_' + str(k))

In [28]:
create_dataframe_per_periode(2)

In [29]:
create_dataframe_per_periode(4)

In [30]:
create_dataframe_per_periode(8)

In [31]:
create_dataframe_per_periode(3)

In [32]:
create_dataframe_per_periode(16)

In [33]:
create_dataframe_per_periode(48)

In [34]:
create_dataframe_per_periode(24)

In [35]:
create_dataframe_per_periode(12)

In [36]:
create_dataframe_per_periode(6)